In [138]:
import json

In [139]:
import pandas
import pendulum

In [140]:
f = open('test.json')

In [141]:
js = json.load(f)

In [142]:
type(js)

TypeError: 'str' object is not callable

In [143]:
js

{'tickets': [],
 'terminus': [{'codes': [{'type': 'source', 'value': '87681007'},
    {'type': 'uic', 'value': '87681007'}],
   'name': 'Corbeil-Essonnes',
   'links': [],
   'coord': {'lat': '48.614405', 'lon': '2.47376'},
   'label': 'Corbeil-Essonnes (Corbeil-Essonnes)',
   'timezone': 'Europe/Paris',
   'id': 'stop_area:SNCF:87681007'},
  {'codes': [{'type': 'source', 'value': '87164780'},
    {'type': 'uic', 'value': '87164780'}],
   'name': 'Stade de France Saint-Denis',
   'links': [],
   'coord': {'lat': '48.91779', 'lon': '2.35134'},
   'label': 'Stade de France Saint-Denis (Saint-Denis)',
   'timezone': 'Europe/Paris',
   'id': 'stop_area:SNCF:87164780'}],
 'links': [{'href': 'https://api.sncf.com/v1/coverage/sncf/journeys?to=stop_area%3ASNCF%3A87271304&datetime_represents=departure&forbidden_uris%5B%5D=line%3ASNCF%3AB&from=stop_area%3ASNCF%3A87271007&datetime=20220703T185301',
   'type': 'next',
   'rel': 'next',
   'templated': False},
  {'href': 'https://api.sncf.com/v1/co

In [144]:
js["journeys"]

[{'status': '',
  'distances': {'taxi': 0,
   'car': 0,
   'walking': 0,
   'bike': 0,
   'ridesharing': 0},
  'links': [{'href': 'https://api.sncf.com/v1/coverage/sncf/journeys?first_section_mode%5B%5D=walking&direct_path=none&to=stop_area%3ASNCF%3A87271304&is_journey_schedules=True&datetime=20220703T185300&last_section_mode%5B%5D=walking&min_nb_journeys=5&min_nb_transfers=2&forbidden_uris%5B%5D=line%3ASNCF%3AB&allowed_id%5B%5D=stop_point%3ASNCF%3A87271411%3ABus&allowed_id%5B%5D=stop_point%3ASNCF%3A87271023%3ARapidTransit&allowed_id%5B%5D=stop_point%3ASNCF%3A87164780%3ABus&allowed_id%5B%5D=stop_point%3ASNCF%3A87164780%3ARapidTransit&allowed_id%5B%5D=stop_point%3ASNCF%3A87271304%3ABus&from=stop_area%3ASNCF%3A87271007',
    'type': 'journeys',
    'rel': 'same_journey_schedules',
    'templated': False},
   {'href': 'https://api.sncf.com/v1/coverage/sncf/journeys?count=1&first_section_mode%5B%5D=walking&direct_path=none&to=stop_area%3ASNCF%3A87271304&is_journey_schedules=True&datetime=2

In [145]:
pendulum.parse(js["journeys"][0]["sections"][1]["departure_date_time"]).to_time_string()

'18:53:00'

In [146]:
js["journeys"][0]["sections"][1]["from"]["name"]

'Paris Gare du Nord Souterrain (Paris)'

In [147]:
js["journeys"][0]["sections"][1]["to"]["name"]

'Stade de France Saint-Denis (Saint-Denis)'

In [148]:
js["journeys"][0]["sections"][1]["type"]

'public_transport'

In [149]:
pendulum.parse(js["journeys"][0]["sections"][1]["arrival_date_time"]).to_time_string()

'18:57:00'

In [189]:

di = {}
for j in range(len(js["journeys"])):
    li = {}

    for i in range(len(js["journeys"][j]["sections"])):
        if "display_informations" in js["journeys"][j]["sections"][i]:
            type = js["journeys"][j]["sections"][i]["display_informations"]["physical_mode"] + \
                   " "  + js["journeys"][j]["sections"][i]["display_informations"]["name"] + \
                   " direction : " + js["journeys"][j]["sections"][i]["display_informations"]["direction"] + " " + pendulum.duration(seconds=js["journeys"][j]["sections"][i]["duration"]).in_words(locale="fr") + " départ à : " +pendulum.parse(js["journeys"][j]["sections"][i]["departure_date_time"]).to_time_string()

        else:
            type = js["journeys"][j]["sections"][i]["type"] + " "

        if js["journeys"][j]["sections"][i]["type"] == "waiting":
            li[i] = "Patientez : " + pendulum.duration(seconds=js["journeys"][j]["sections"][i]["duration"]).in_words(locale="fr")
        else :
            li[i] = (js["journeys"][j]["sections"][i]["from"]["name"] + " -> " + js["journeys"][j]["sections"][i]["to"]["name"] +" via : " + type)
    di[j] = li
    di[j]["departure_date_time"] = pendulum.parse(js["journeys"][j]["departure_date_time"]).to_time_string()
    di[j]["arrival_date_time"] = pendulum.parse(js["journeys"][j]["arrival_date_time"]).to_time_string()
    di[j]["duration"] = pendulum.duration(seconds=js["journeys"][j]["duration"]).in_words()


In [190]:
di[0]

{0: 'Paris Gare du Nord (Paris) -> Paris Gare du Nord Souterrain (Paris) via : marche à pied ',
 1: 'Paris Gare du Nord Souterrain (Paris) -> Stade de France Saint-Denis (Saint-Denis) via : RER / Transilien D direction : Villiers-le-Bel - Gonesse (Arnouville) 4 minutes départ à : 18:53:00',
 2: 'Stade de France Saint-Denis (Saint-Denis) -> Stade de France Saint-Denis (Saint-Denis) via : transfer ',
 3: 'Patientez : 3 minutes',
 4: 'Stade de France Saint-Denis (Saint-Denis) -> Aulnay-sous-Bois (Aulnay-sous-Bois) via : Bus B Bus direction : Aéroport Charles de Gaulle 1 (Tremblay-en-France) 50 minutes départ à : 19:05:00',
 5: 'Patientez : 5 minutes',
 6: 'Aulnay-sous-Bois (Aulnay-sous-Bois) -> La Courneuve - Aubervilliers (La Courneuve) via : Bus B Bus direction : La Plaine Stade France (Saint-Denis) 35 minutes départ à : 20:00:00',
 7: 'La Courneuve - Aubervilliers (La Courneuve) -> La Courneuve - Aubervilliers (La Courneuve) via : marche à pied ',
 'departure_date_time': '18:53:00',
 '

In [171]:
pendulum.parse(js["journeys"][0]["arrival_date_time"]).to_time_string()


'20:35:00'

In [172]:
pendulum.parse(js["journeys"][0]["departure_date_time"]).to_time_string()

'18:53:00'

In [174]:
pendulum.duration(seconds=js["journeys"][0]["duration"]).in_words()

'1 hour 42 minutes'

In [196]:
for i in range(len(di)):
    print("Trajet n°" + str(i))
    for j in range(len(di[i]) - 3):
        print(di[i][j])

Trajet n°0
Paris Gare du Nord (Paris) -> Paris Gare du Nord Souterrain (Paris) via : marche à pied 
Paris Gare du Nord Souterrain (Paris) -> Stade de France Saint-Denis (Saint-Denis) via : RER / Transilien D direction : Villiers-le-Bel - Gonesse (Arnouville) 4 minutes départ à : 18:53:00
Stade de France Saint-Denis (Saint-Denis) -> Stade de France Saint-Denis (Saint-Denis) via : transfer 
Patientez : 3 minutes
Stade de France Saint-Denis (Saint-Denis) -> Aulnay-sous-Bois (Aulnay-sous-Bois) via : Bus B Bus direction : Aéroport Charles de Gaulle 1 (Tremblay-en-France) 50 minutes départ à : 19:05:00
Patientez : 5 minutes
Aulnay-sous-Bois (Aulnay-sous-Bois) -> La Courneuve - Aubervilliers (La Courneuve) via : Bus B Bus direction : La Plaine Stade France (Saint-Denis) 35 minutes départ à : 20:00:00
La Courneuve - Aubervilliers (La Courneuve) -> La Courneuve - Aubervilliers (La Courneuve) via : marche à pied 
